In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from attn_gym import visualize_attention_scores
from torch.nn.attention.flex_attention import flex_attention, create_block_mask
from torch.nn.attention.flex_attention import _mask_mod_signature
from torch.nn.attention.flex_attention import (
    _score_mod_signature,
    _mask_mod_signature,
    _vmap_for_bhqkv,
    _ModificationType,
)
from torch.nn.utils.rnn import pad_sequence
from typing import List, Union
from torch import Tensor
import torch
import random

def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

def _offsets_to_doc_ids_tensor(offsets):
    device = offsets.device
    offsets = offsets[offsets != -1]
    counts = offsets[1:] - offsets[:-1]
    return torch.repeat_interleave(
        torch.arange(len(counts), device=device, dtype=torch.int32), counts
    )


def length_to_offsets(lengths: List[int], device: Union[str, torch.device]) -> Tensor:
    """Converts a list of lengths to a list of offsets.

    Args:
        lengths: A list of lengths.

    """
    offsets = [0]
    offsets.extend(lengths)
    offsets = torch.tensor(offsets, device=device, dtype=torch.int32)
    offsets = torch.cumsum(offsets, dim=-1)
    return offsets


def generate_doc_mask_mod(mask_mod: _mask_mod_signature, offsets: Tensor) -> _mask_mod_signature:
    """Generates mask mods that apply to inputs to flex attention in the sequence stacked
    format.

    Args:
        mask_mod: The mask mod to apply to the documents
        offsets: This tensor should be of shape(num_documents + 1)
            this should contain the cumulative counts of document tokens.
            e.g. if you have 3 documents of length 2, 4, 3 then
            offsets = [0, 2, 6, 9]

    Note:
        What is the sequence stacked format? When assembling batches of inputs, we
        take multiple sequences and stack them together to form 1 large sequence. We then
        use masking to ensure that the attention scores are only applied to tokens within
        the same document.
    """
    offsets = pad_sequence(offsets, batch_first = True, padding_value = -1)
    document_ids = [_offsets_to_doc_ids_tensor(offsets[i]) for i in range(offsets.shape[0])]
    document_ids = torch.stack(document_ids, 0)

    def doc_mask_mod(b, h, q_idx, kv_idx):
        same_doc = document_ids[b][q_idx] == document_ids[b][kv_idx]
        q_logical = q_idx - offsets[b][document_ids[b][q_idx]]
        kv_logical = kv_idx - offsets[b][document_ids[b][kv_idx]]
        inner_mask = mask_mod(b, h, q_logical, kv_logical)
        return same_doc & inner_mask

    return doc_mask_mod

In [3]:
flex_attention = torch.compile(flex_attention, dynamic = False)
create_block_mask = torch.compile(create_block_mask, dynamic = False)

In [4]:
device = 'cuda'

In [5]:
def generate_random_lengths(total_length, num_documents):
    # Initialize all lengths to 1 to ensure each document has at least one token
    lengths = [1] * num_documents
    remaining_length = total_length - num_documents

    # Randomly distribute the remaining length
    for _ in range(remaining_length):
        index = random.randint(0, num_documents - 1)
        lengths[index] += 1

    return lengths

max_seq_len, doc_count = 21, 4
B, H, SEQ_LEN, HEAD_DIM = 2, 1, max_seq_len, 128

offsets = []
for i in range(doc_count, doc_count + B, 1):
    lengths = generate_random_lengths(max_seq_len, i)

    offsets.append(length_to_offsets(lengths, device))

def make_tensor():
    return torch.randn(B, H, SEQ_LEN, HEAD_DIM, device=device)

q, k, v = make_tensor(), make_tensor(), make_tensor()

In [6]:
def forward(q, k, v, offsets):
    document_causal_mask = generate_doc_mask_mod(causal_mask, offsets)
    block_mask = create_block_mask(document_causal_mask, None, None, SEQ_LEN, SEQ_LEN, device="cuda", _compile=True)
    flex = flex_attention(q, k, v, block_mask=block_mask)
    return flex

In [10]:
%%time

flex = forward(q, k, v, offsets)

CPU times: user 2.54 ms, sys: 228 µs, total: 2.77 ms
Wall time: 2.08 ms


In [43]:
offsets = []
extra = 2
for i in range(doc_count, doc_count + B + extra, extra):
    lengths = generate_random_lengths(max_seq_len, i)

    offsets.append(length_to_offsets(lengths, device))
offsets

[tensor([ 0,  8, 13, 17, 21], device='cuda:0'),
 tensor([ 0,  4,  7, 10, 15, 18, 21], device='cuda:0')]

In [44]:
%%time

flex = forward(q, k, v, offsets)

CPU times: user 1.87 ms, sys: 365 µs, total: 2.23 ms
Wall time: 1.73 ms


In [32]:
def block_diagonal_concat_inverted(*masks, dtype=torch.bfloat16):
    total_size = sum(mask.size(0) for mask in masks)
    combined_mask = torch.zeros(total_size, total_size, dtype=dtype)

    current_pos = 0

    for mask in masks:
        size = mask.size(0)
        combined_mask[current_pos:current_pos + size, current_pos:current_pos + size] = mask
        current_pos += size

    min_value = torch.finfo(dtype).min if dtype.is_floating_point else torch.iinfo(dtype).min
    inverted_mask = torch.where(combined_mask == 1, torch.tensor(0, dtype=dtype), min_value)
    return inverted_mask.unsqueeze(0)

In [45]:
%%time

masks = []
for i in range(len(offsets)):
    masks_ = []
    masking = torch.diff(offsets[i])
    for m in masking:
        masks_.append(torch.tril(torch.ones(m, m)))
    
    masks.append(block_diagonal_concat_inverted(*masks_, dtype = q.dtype))
    
masks = torch.stack(masks, 0).to('cuda')

CPU times: user 744 ms, sys: 334 µs, total: 744 ms
Wall time: 72.8 ms


In [46]:
%%time

sdpa = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask = masks)

CPU times: user 472 µs, sys: 84 µs, total: 556 µs
Wall time: 292 µs


In [47]:
sdpa.argmax(-1)

tensor([[[ 71,  71, 115,  71,  49,  14,  71,  50,  55,  53,  55,  94,  97,  99,
            0,  20, 113,  75, 120,  86, 118]],

        [[ 54,  54,  54,  54,  93,  83,  14,  82,  73,  58, 107,  35,  35,  52,
           35,  16,   4,  72,   1,  53,   0]]], device='cuda:0')

In [48]:
flex.argmax(-1)

tensor([[[ 71,  71, 115,  71,  49,  14,  71,  50,  55,  53,  55,  94,  97,  99,
            0,  20, 113,  75, 120,  86, 118]],

        [[ 54,  54,  54,  54,  93,  83,  14,  82,  73,  58, 107,  35,  35,  52,
           35,  16,   4,  72,   1,  53,   0]]], device='cuda:0')

In [49]:
sdpa.argmax(-1) == flex.argmax(-1)

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True, True, True, True, True, True]]],
       device='cuda:0')